In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats

In [2]:
cd ..

/Users/alphonsowoodbury/DS/mod2project/flatiron_mod2_project_kch


In [82]:
#importing data
df = pd.read_csv('data/kc_house_data.csv')

In [83]:
#data cleaning:
#changing question marks to 0.0
df = df.replace('?', 0.0)
df.view = df.view.replace(np.nan, 0)
#changing all column object types to floats (except date column)
df.loc[:, df.columns != 'date'] = df.loc[:,df.columns != 'date'].astype('float')

#changing all 0.0 in sqft_basement column, yr_renovated, and waterfront columns to NaN values
df['sqft_basement'] = df['sqft_basement'].replace(0.0 , np.nan)
df['waterfront'] = df.waterfront.replace(0.0, np.nan)
df['yr_renovated'] =df['yr_renovated'].replace(0.0, np.nan)
#changing date column to datetime values
df['date'] = pd.to_datetime(df['date'])

In [84]:
#Feature Engineering:
#creating has_basement, has_waterfront, has_renovation columns with True/False values
df['has_waterfront'] = df['waterfront'].notnull().astype('int64')
df['has_basement'] = df['sqft_basement'].notnull().astype('int64')
df['has_renovation'] = df['yr_renovated'].notnull().astype('int64')
#creating eff_built column (which updates built year depending on whether it was renovated or not)
df.loc[df['yr_renovated'].notnull(), 'eff_built'] = 2020 - df['yr_renovated']
df.loc[df['yr_renovated'].isnull(), 'eff_built'] = 2020 -df['yr_built']
df.eff_built = df.eff_built.astype('int64')
df.bedrooms = df.bedrooms.astype('int64')
df.bathrooms = df.bathrooms.astype('int64')
df.floors = df.floors.astype('int64')
df.zipcode = df.zipcode.astype('int64')
df.condition = df.condition.astype('int64')
df.grade = df.grade.astype('int64')
df.view = df.view.astype('int64')
df = df.drop(['id'],axis=1)

In [85]:
# #Creating new dataframe with desired columns:
# df = df[['id','date','price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','condition','grade',\
#          'yr_built','zipcode','sqft_living15','sqft_lot15','has_basement','has_waterfront','has_renovation',\
#          'eff_built']]
#holding off removing columns

In [86]:
df.to_csv('kc_cleaned.csv',index=False)
df

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,has_waterfront,has_basement,has_renovation,eff_built
0,2014-10-13,221900.0,3,1,1180.0,5650.0,1,NaN,0,3,...,NaN,98178,47.5112,-122.257,1340.0,5650.0,0,0,0,65
1,2014-12-09,538000.0,3,2,2570.0,7242.0,2,NaN,0,3,...,1991.0,98125,47.7210,-122.319,1690.0,7639.0,0,1,1,29
2,2015-02-25,180000.0,2,1,770.0,10000.0,1,NaN,0,3,...,NaN,98028,47.7379,-122.233,2720.0,8062.0,0,0,0,87
3,2014-12-09,604000.0,4,3,1960.0,5000.0,1,NaN,0,5,...,NaN,98136,47.5208,-122.393,1360.0,5000.0,0,1,0,55
4,2015-02-18,510000.0,3,2,1680.0,8080.0,1,NaN,0,3,...,NaN,98074,47.6168,-122.045,1800.0,7503.0,0,0,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,2014-05-21,360000.0,3,2,1530.0,1131.0,3,NaN,0,3,...,NaN,98103,47.6993,-122.346,1530.0,1509.0,0,0,0,11
21593,2015-02-23,400000.0,4,2,2310.0,5813.0,2,NaN,0,3,...,NaN,98146,47.5107,-122.362,1830.0,7200.0,0,0,0,6
21594,2014-06-23,402101.0,2,0,1020.0,1350.0,2,NaN,0,3,...,NaN,98144,47.5944,-122.299,1020.0,2007.0,0,0,0,11
21595,2015-01-16,400000.0,3,2,1600.0,2388.0,2,NaN,0,3,...,NaN,98027,47.5345,-122.069,1410.0,1287.0,0,0,0,16


In [75]:
# #Correlation Heatmap
# df_corr = df.corr()
# sns.set(rc={'figure.figsize':(20,10)})
# ax = sns.heatmap(df_corr, center=0, vmin=-1, vmax=1, linewidth = 5, annot=True, cmap = 'YlGnBu' )
# bottom, top = ax.get_ylim()
# ax.set_ylim(bottom + 0.5, top - 0.5)
# plt.show()

In [76]:
# pd.plotting.scatter_matrix(df,figsize  = [15, 15]);
# plt.show()